In [8]:
import csv 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [19]:
schools= "Resources/schools_complete.csv"
students = "Resources/students_complete.csv"


school_df = pd.read_csv(schools)

school_df.columns

school = school_df.rename(columns={"school_name" : "School"})
school.head()





,School ID,School,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [21]:
student_df = pd.read_csv(students)

student_df.columns

student = student_df.rename(columns={'school_name' : 'School'})
student.head()

,Student ID,student_name,gender,grade,School,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [22]:
complete_data = pd.merge(school, student, on=('School'))
complete_data.head()


,School ID,School,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [11]:
total_schools = school_df["School ID"].count()
total_schools

15

In [54]:
total_students = student_df["Student ID"].count()
total_students

39170

In [43]:
total_budget = school_df["budget"].sum()
total_budget

24649428

In [44]:
average_math = student_df["math_score"].mean()
average_math

78.98537145774827

In [45]:
average_reading = student_df["reading_score"].mean()
average_reading

81.87784018381414

In [46]:
pass_math = ((student_df['math_score']>=70).sum()/total_students)*100
pass_math

74.9808526933878

In [47]:
pass_reading = ((student_df['reading_score']>=70).sum()/total_students)*100
pass_reading

85.80546336482001

In [48]:
pass_overall = ((pass_math+pass_reading)/2)
pass_overall

80.39315802910392

In [49]:
overview_df = pd.DataFrame({
    "Total Schools":[total_schools],
    "Total Budget":[total_budget],
    "Total Students":[total_students],
    "Avg Math Score":[average_math],
    "Avg Reading Score":[average_reading],
    "Percentage Passing Math":[pass_math],
    "Percentage Passing Reading":[pass_reading],
    "Percentage Passing Overall":[pass_overall]
})
overview_df

,Total Schools,Total Budget,Total Students,Avg Math Score,Avg Reading Score,Percentage Passing Math,Percentage Passing Reading,Percentage Passing Overall
0,15,24649428,39170,78.985371,81.87784,74.980853,85.805463,80.393158


In [35]:
school_types = school.set_index(["School"])["type"]

per_school_counts = complete_data["School"].value_counts()

per_school_budget = complete_data.groupby(["School"]).mean()["budget"]
per_student_budget = per_school_budget/ per_school_counts

avg_math_score = complete_data.groupby(["School"]).mean()["math_score"]
avg_reading_score = complete_data.groupby(["School"]).mean()["reading_score"]

school_passing_math =  complete_data[complete_data["math_score"] > 70].groupby("School").count()["student_name"]
school_passing_reading =  complete_data[complete_data["reading_score"] > 70].groupby("School").count()["student_name"]

percent_passing_math = school_passing_math / per_school_counts * 100
percent_passing_reading = school_passing_reading / per_school_counts * 100
overall_passing_rate = (percent_passing_math + percent_passing_reading) / 2

School_wise_Summary = pd.DataFrame({"School Type":school_types, "Total Students":per_school_counts,
                               "Total School Budget":per_school_budget, "Per Student Budget":per_student_budget,
                                "Average Math Score":avg_math_score, "Average Reading Score":avg_reading_score,
                                "% Passing Math":percent_passing_math, "% Passing Reading":percent_passing_reading,
                                 "Overall Passing Rate":overall_passing_rate})            


School_wise_Summary = School_wise_Summary[["School Type","Total Students","Total School Budget","Per Student Budget",
                                "Average Math Score","Average Reading Score","% Passing Math","% Passing Reading",
                                "Overall Passing Rate"]]

School_wise_Summary["Total School Budget"] = School_wise_Summary["Total School Budget"].map("${:,.2f}".format)
School_wise_Summary["Per Student Budget"] = School_wise_Summary["Per Student Budget"].map("${:,.2f}".format)


School_wise_Summary.head()


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,64.630225,79.300643,71.965434
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,65.753925,77.510040,71.631982
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,89.713896,93.392371,91.553134


In [36]:
Top_Schools = School_wise_Summary.sort_values(["Overall Passing Rate"], ascending = False)
Top_Schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,90.932983,93.254490,92.093736
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,91.683992,92.203742,91.943867
Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.682222,83.955000,90.277778,93.444444,91.861111
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,91.686183


In [38]:
Bottom_Schools = School_wise_Summary.sort_values(["Overall Passing Rate"], ascending = True)
Bottom_Schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,64.066017,77.744436,70.905226
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,63.852132,78.281874,71.067003
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,71.467098


In [48]:
nineth_grade_score =  complete_data[complete_data["grade"] == "9th"].groupby("School").mean()["math_score"]
tenth_grade_score =  complete_data[complete_data["grade"] == "10th"].groupby("School").mean()["math_score"]
eleventh_grade_score =  complete_data[complete_data["grade"] == "11th"].groupby("School").mean()["math_score"]
twelveth_grade_score =  complete_data[complete_data["grade"] == "12th"].groupby("School").mean()["math_score"]

math_score_by_grade_df = pd.DataFrame({"9th":nineth_grade_score, "10th":tenth_grade_score,
                               "11th":eleventh_grade_score,"12th":twelveth_grade_score})            

Math_Score_by_Grade = math_score_by_grade_df[["9th", "10th", "11th", "12th"]]
Math_Score_by_Grade.index.name = None

Math_Score_by_Grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


In [49]:

nineth_grade_score =  complete_data[complete_data["grade"] == "9th"].groupby("School").mean()["reading_score"]
tenth_grade_score =  complete_data[complete_data["grade"] == "10th"].groupby("School").mean()["reading_score"]
eleventh_grade_score =  complete_data[complete_data["grade"] == "11th"].groupby("School").mean()["reading_score"]
twelveth_grade_score =  complete_data[complete_data["grade"] == "12th"].groupby("School").mean()["reading_score"]

reading_score_by_grade_df = pd.DataFrame({"9th":nineth_grade_score, "10th":tenth_grade_score,
                               "11th":eleventh_grade_score,"12th":twelveth_grade_score})            

Reading_Score_by_Grade = reading_score_by_grade_df[["9th", "10th", "11th", "12th"]]
Reading_Score_by_Grade.index.name = None

Reading_Score_by_Grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


In [50]:
spending_bins = [0, 585, 615, 645, 675]
spending_ranges = ["<585", "585-615", "615-645", "645-675"]

School_wise_Summary["Spending Ranges (Per Student)"] = pd.cut(per_student_budget, spending_bins, labels = spending_ranges)


spending_math_score = School_wise_Summary.groupby(["Spending Ranges (Per Student)"]).mean()['Average Math Score']
Spending_reading_score = School_wise_Summary.groupby(["Spending Ranges (Per Student)"]).mean()['Average Reading Score']
spending_passing_math =  School_wise_Summary.groupby(["Spending Ranges (Per Student)"]).mean()['% Passing Math']
spending_passing_reading =  School_wise_Summary.groupby(["Spending Ranges (Per Student)"]).mean()['% Passing Reading']
overall_passing_rate =  (spending_math_score + Spending_reading_score) / 2


Spending_Score = pd.DataFrame({"Average Math Score":spending_math_score, "Average Reading Score":Spending_reading_score,
                               "% Passing Math":spending_passing_math,"% Passing Reading":spending_passing_reading,
                                    "Overall Passing Rate":overall_passing_rate}) 


Spending_Score = Spending_Score[["Average Math Score", "Average Reading Score",
                               "% Passing Math","% Passing Reading","Overall Passing Rate"]]


Spending_Score

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<585,83.455399,83.933814,90.350436,93.325838,83.694607
585-615,83.599686,83.885211,90.788049,92.410786,83.742449
615-645,79.079225,81.891436,73.021426,83.214343,80.485330
645-675,76.997210,81.027843,63.972368,78.427809,79.012526


In [51]:

size_bins = [0, 1000, 2000, 5000]
size_ranges = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
School_wise_Summary["School Size"] = pd.cut(School_wise_Summary["Total Students"], size_bins, labels = size_ranges)

avg_math_score = School_wise_Summary.groupby(["School Size"]).mean()['Average Math Score']
avg_reading_score = School_wise_Summary.groupby(["School Size"]).mean()['Average Reading Score']
percent_passing_math =  School_wise_Summary.groupby(["School Size"]).mean()['% Passing Math']
percent_passing_reading =  School_wise_Summary.groupby(["School Size"]).mean()['% Passing Reading']
overall_passing_rate = School_wise_Summary.groupby(["School Size"]).mean()['Overall Passing Rate']

Size_Score = pd.DataFrame({"Average Math Score":avg_math_score, "Average Reading Score":avg_reading_score,
                               "% Passing Math":percent_passing_math,"% Passing Reading":percent_passing_reading,
                                    "Overall Passing Rate":overall_passing_rate})   


Size_Score = Size_Score[["Average Math Score", "Average Reading Score",
                               "% Passing Math","% Passing Reading","Overall Passing Rate"]]


Size_Score

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,91.158155,92.471895,91.815025
Medium (1000-2000),83.374684,83.864438,89.931303,93.244843,91.588073
Large (2000-5000),77.746417,81.344493,67.631335,80.190800,73.911067


In [52]:
avg_math_score = School_wise_Summary.groupby(["School Type"]).mean()['Average Math Score']
avg_reading_score = School_wise_Summary.groupby(["School Type"]).mean()['Average Reading Score']
percent_passing_math =  School_wise_Summary.groupby(["School Type"]).mean()['% Passing Math']
percent_passing_reading =  School_wise_Summary.groupby(["School Type"]).mean()['% Passing Reading']
overall_passing_rate = School_wise_Summary.groupby(["School Type"]).mean()['Overall Passing Rate']


Type_Score = pd.DataFrame({"Average Math Score":avg_math_score, "Average Reading Score":avg_reading_score,
                               "% Passing Math":percent_passing_math,"% Passing Reading":percent_passing_reading,
                                    "Overall Passing Rate":overall_passing_rate})            

Type_Score = Type_Score[["Average Math Score", "Average Reading Score",
                               "% Passing Math","% Passing Reading","Overall Passing Rate"]]

Type_Score

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,90.363226,93.052812,91.708019
District,76.956733,80.966636,64.302528,78.324559,71.313543
